In [17]:
import os, sys, time, math

module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *
import gym, envs
#env = gym.make("SmallMaze-v0")
#env.render()
#env.actions

## Assignment 1: Breadth-First Search (BFS)

Your first assignment is to implement the BFS algorithm on SmallMaze. In particular, you should implement both *tree search* and *graph search* versions of BFS that the generic bfs will call. 

The results returned by your **BFS** must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

**For the time_cost, we consider a node checked after its generation!**

After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- BFS_TreeSearch(problem)
- BFS_GraphSearch(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for *Graph Search* and *Tree Search*:

<img src="images/tree-graph-search.png" width="600">

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *BFS* algorithm, note that it refers to the implementation of the *Graph Search Version*:

<img src="images/BFS2.png" width="600">

**The next two functions have to be implemented:**

In [18]:
def BFS_TreeSearch(problem):
    """
    Tree Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)# No parent node 
    time_cost = 1
    space_cost = 1
    # space cost is the max (max len in the frontier)
    
    goalState = problem.goalstate
    if goalState == node.state :
        return build_path(node), time_cost, space_cost
    frontier = NodeQueue()
    frontier.add(node)

    while not frontier.is_empty():
        if (len(frontier)) > space_cost:
            space_cost = len(frontier)
            
        parent = frontier.remove()
        
        for action in problem.actions:
            child = Node(problem.sample(parent.state,action),parent)
            time_cost+=1
            if child not in frontier: 
                if  goalState ==child.state:
                    return build_path(child), time_cost, space_cost
                frontier.add(child)

    return "failure", time_cost, space_cost

    #return build_path(node), time_cost, space_cost

In [19]:
def BFS_GraphSearch(problem):
    """
    Graph Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    node = Node(problem.startstate, None)# No parent node 
    time_cost = 1
    space_cost = 1
    goalState = problem.goalstate
    
    if goalState == node.state :
        return build_path(node), time_cost, space_cost
    frontier = NodeQueue()
    frontier.add(node)
    explored = []
    #frontierState = []
    #frontierState.append(node.state)
    
    while not frontier.is_empty():
        if (len(frontier)+len(explored)) > space_cost:
            space_cost = len(frontier) + len(explored)
        
        parent = frontier.remove()
        #frontierState.remove(parent.state)
        explored.append(parent.state)

        for action in problem.actions:
            child = Node(problem.sample(parent.state,action),parent)
            time_cost+=1
            if (child.state not in frontier) and (child.state not in explored) :
                    if goalState == child.state:
                        print(build_path(child))
                        return build_path(child), time_cost, space_cost
                    frontier.add(child)
                    #frontierState.append(child.state)

    return "failure", time_cost, space_cost

    #return build_path(node), time_cost, space_cost       

**The following code calls your tree search and graph search version of BFS and checks the results**

In [20]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = BFS_TreeSearch(environment)
solution_gs, time_gs, memory_gs = BFS_GraphSearch(environment)

results = CheckResult_L1A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

(1, 0, 4, 8, 12, 16, 17, 18, 19)
##########################################
#######  BFS TREE SEARCH PROBLEM  ########
##########################################
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 103723
Max n° of nodes in memory: 77791

===> Your solution is correct!

##########################################
#######  BFS Graph SEARCH PROBLEM  #######
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 59
Max n° of nodes in memory: 15

===> Your solution is correct!



## Assignment 2:  Depth-Limited Search (DLS) and Iterative Deepening depth-first Search (IDS)

Your second assignment is to implement the IDS algorithm on SmallMaze. 
In particular, you are required to implement *DLS* in the *graph search* version, *DLS* in the *tree search* version, and the final *Iterative_DLS*.

Similarly to assignment 1, the results returned by your ids must be in the following form (path, Time Cost, Space Cost) described above. After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- Recursive_DLS_TreeSearch(node, problem, limit)
- Recursive_DLS_GraphSearch(node, problem, limit, explored)
- IDS(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *Depth-Limited Search (Tree Search Version)* and *Iterative deepening depth-first search (Tree Search Version)*:
<img src="images/dls.png" width="600">
<img src="images/ids.png" width="600">

#### Note that Node() has a depthcost variable that represents the depth of the node in the search tree. This variable is automatically set by the Node constructor: if the root node has a depthcost of 0, its children will have a depthcost increased by 1. An example is shown below. The depthcost is useful to compute the space cost for IDS. An example is shown below:

In [21]:
start = environment.startstate
root = Node(start)  # parent = None and depthcost = 0 as default

child = Node(environment.sample(start, 0), root) # the depthcost is set automatically in the Node constructor
print("Root depthcost: {}\tChild depthcost: {}".format(root.depthcost, child.depthcost))

Root depthcost: 0	Child depthcost: 1


In [22]:
# Inutile ???? Deep Limited Search 
def DLS(problem, limit, RDLS_Function): # Uses one of the 2 versions of recursive deep limited search 
      
    node = Node(problem.startstate)
    return RDLS_Function(node, problem, limit, set())

**The next two functions have to be implemented:**

In [63]:
def Recursive_DLS_GraphSearch(node, problem, limit, explored): # Deptch limited search with recursion ! 
    """
    DLS (Tree Search Version)
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    # metto time cost = 1 solo all'inizio quando il limite è al massimo 
    #if node.parent == None:
    time_cost = 1 
    space_cost = node.depthcost
    if(node.state not in explored):
        explored.add(node.state)
    
    goal = problem.goalstate
    if node.state == goal:  # trovato obbiettivo 
        return build_path(node),time_cost, space_cost
    elif limit==0 : 
        return "cut_off", time_cost, space_cost
    else:
        cutOff = False
        for action in problem.actions:
            #time_cost+=1
            child = Node(problem.sample(node.state,action),node)
            if(child.state not in explored):
                result,tc,sc = Recursive_DLS_TreeSearch(child,problem,limit-1) 
                time_cost += tc 
                space_cost= max(sc,space_cost)
                if result == "cut_off": # cutoff solo di questa linea 
                    cutOff = True 
                elif result != "failure": # Corretto 
                    return result, time_cost, space_cost
            else: 
                cutOff = True 
        if cutOff: # Cutoff 
            return "cut_off", time_cost, space_cost

In [58]:
def Recursive_DLS_TreeSearch(node, problem, limit, explored=None): # Deptch limited search with recursion ! 
    """
    DLS (Tree Search Version)
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    # metto time cost = 1 solo all'inizio quando il limite è al massimo 
    #if node.parent == None:
    time_cost = 1 
    space_cost = node.depthcost
    
    goal = problem.goalstate
    if node.state == goal:  # trovato obbiettivo 
        return build_path(node),time_cost, space_cost
    
    elif limit==0 : 
        return "cut_off", time_cost, space_cost
    else:
        cutOff = False
        for action in problem.actions:
            #time_cost+=1
            child = Node(problem.sample(node.state,action),node)
            result,tc,sc = Recursive_DLS_TreeSearch(child,problem,limit-1) 
            time_cost += tc 
            space_cost= max(sc,space_cost)
            if result == "cut_off": # cutoff solo di questa linea 
                cutOff = True 
            elif result != "failure": # Corretto 
                return result, time_cost, space_cost
        if cutOff: # Cutoff 
            return "cut_off", time_cost, space_cost

In [65]:
def IDS(problem, DLS_Function): # iterative deepening search 
    """
    Iteartive_DLS DLS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    total_time_cost = 0
    total_space_cost = 1
    explored = set()
    
    for i in zero_to_infinity():
        result,tc,sc = DLS_Function(root,problem,i,explored)
        total_time_cost+= tc
        total_space_cost = max(total_space_cost,sc)
        if (result != "cut_off"):
            return result, total_time_cost, total_space_cost, i

**The following code calls your version of IDS and checks the results:**

In [66]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts, iterations_ts = IDS(environment, Recursive_DLS_TreeSearch)
solution_gs, time_gs, memory_gs, iterations_gs = IDS(environment, Recursive_DLS_GraphSearch)

results = CheckResult_L1A2([solution_ts, time_ts, memory_ts, iterations_ts], [solution_gs, time_gs, memory_gs, iterations_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  IDS TREE SEARCH PROBLEM  ########
##########################################
Necessary Iterations: 9
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 138298
Max n° of nodes in memory: 9

===> Your solution is correct!

##########################################
#######  IDS GRAPH SEARCH PROBLEM  #######
##########################################
Necessary Iterations: 9
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 80050
Max n° of nodes in memory: 9

> Your necessary iterations are not correct, should be: 11



### Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?